# ChatGLM3基于LoRA实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("/root/autodl-tmp/weitiao/data/alpaca_data_zh")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[:1]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'],
 'input': [''],
 'instruction': ['保持健康的三个提示。']}

## Step3 数据集预处理

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/chatglm3-6b-base", trust_remote_code=True)
tokenizer

ChatGLMTokenizer(name_or_path='/root/autodl-tmp/chatglm3-6b-base', vocab_size=64798, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=False)

In [4]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = "\n".join([example["instruction"], example["input"]]).strip()     # prompt
    instruction = tokenizer.build_chat_input(instruction, history=[], role="user")  # [gMASK]sop<|user|> \n prompt <|assistant|>
    response = tokenizer("\n" + example["output"], add_special_tokens=False)        # \n response
    input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id] #eos token
    attention_mask = instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [5]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'[gMASK]sop<|user|> \n 解释为什么以下分数等同于1/4\n输入：4/16<|assistant|> \n4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'\n4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。'

## Step4 创建模型

In [6]:
import torch
model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/chatglm3-6b-base",
                                             trust_remote_code=True,
                                             low_cpu_mem_usage=True, 
                                             torch_dtype=torch.half, 
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
for name, param in model.named_parameters():
    print(name)

transformer.embedding.word_embeddings.weight
transformer.encoder.layers.0.input_layernorm.weight
transformer.encoder.layers.0.self_attention.query_key_value.weight
transformer.encoder.layers.0.self_attention.query_key_value.bias
transformer.encoder.layers.0.self_attention.dense.weight
transformer.encoder.layers.0.post_attention_layernorm.weight
transformer.encoder.layers.0.mlp.dense_h_to_4h.weight
transformer.encoder.layers.0.mlp.dense_4h_to_h.weight
transformer.encoder.layers.1.input_layernorm.weight
transformer.encoder.layers.1.self_attention.query_key_value.weight
transformer.encoder.layers.1.self_attention.query_key_value.bias
transformer.encoder.layers.1.self_attention.dense.weight
transformer.encoder.layers.1.post_attention_layernorm.weight
transformer.encoder.layers.1.mlp.dense_h_to_4h.weight
transformer.encoder.layers.1.mlp.dense_4h_to_h.weight
transformer.encoder.layers.2.input_layernorm.weight
transformer.encoder.layers.2.self_attention.query_key_value.weight
transformer.enco

## Lora

### PEFT Step1 配置文件

In [7]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

config = LoraConfig(target_modules=["query_key_value"])
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

### PEFT Step2 创建模型

In [8]:
model = get_peft_model(model, config)

In [ ]:
config

In [ ]:
for name, parameter in model.named_parameters():
    print(name)

In [12]:
model.print_trainable_parameters()

trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.031217444255383614


In [ ]:
model

## Step5 配置训练参数

In [9]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/chatglm3output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    logging_steps=10,
    num_train_epochs=1,
    learning_rate=1e-4,
    adam_epsilon=1e-4,
    remove_unused_columns=False
)

## Step6 创建训练器

In [10]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

## Step7 模型训练

In [11]:
trainer.train()

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,1.917500
20,1.979500
30,1.859300
40,1.831000
50,1.679900
60,1.735100
70,1.685000
80,1.591300
90,1.678700
100,1.566700


TrainOutput(global_step=839, training_loss=1.5791998898457287, metrics={'train_runtime': 5432.5466, 'train_samples_per_second': 4.944, 'train_steps_per_second': 0.154, 'total_flos': 9.76144784608297e+16, 'train_loss': 1.5791998898457287, 'epoch': 1.0})

## Step8 模型推理

In [12]:
model.eval()
print(model.chat(tokenizer, "如何写简历？", history=[])[0])

写简历时，您可以按照以下步骤进行：

1. 个人信息：在开头部分，写上您的姓名、联系方式和地址。

2. 教育背景：列出您所获得的所有学历，包括学校名称、学位和毕业日期。

3. 工作经历：在简历中，按照时间顺序列出您过去的工作经历，包括公司名称、职位、职责和时间。

4. 技能和证书：在您的简历中，列出您掌握的技能和证书，例如编程语言、软件、证书和证书等级。

5. 个人亮点：在您的简历中，列出您的个人亮点，例如您在团队合作、创新、解决问题和沟通方面的优势。

6. 总结：在结尾部分，简要总结您在各个领域的工作经验和技能，并表达您对未来的职业发展的期望。

请注意，简历应该简洁明了，突出您的优势和技能，同时避免使用过于复杂的语言和词汇。您可以参考模板或在线简历生成器来帮助您创建一份简历。
